In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import sys
sys.path.append('/home/azhar04/project/1. dev/quantum-dev/ShotOptimized-ADAPT-VQE/ADAPT-VQE/')
print(sys.executable)

/home/azhar04/anaconda3/envs/qiskit-1-2/bin/python


In [84]:
from src.pools import QE
from src.utilities import to_qiskit_operator
from src.molecules import create_h2, create_h4

from algorithms.adapt_vqe import AdaptVQE

r = 0.742
molecule = create_h4(r)
pool = QE(molecule)

print(pool.get_q_op(0), '\n')
print(type(pool.get_q_op(0)), '\n')
# print(pool)

operator_qiskit = to_qiskit_operator(pool.get_q_op(0))

0.5j [X0 Y2] +
-0.5j [Y0 X2] 

<class 'openfermion.ops.operators.qubit_operator.QubitOperator'> 

None
==TERM== -0.5j [Y0 X2]
pauli string ((0, 'Y'), (2, 'X'))
AA
--qiskit_op-1 SparsePauliOp(['Y'],
              coeffs=[1.+0.j])
AA
--qiskit_op-1 SparsePauliOp(['XIY'],
              coeffs=[1.+0.j])
BB True
0
coefficient -0.5j
qiskit_op SparsePauliOp(['XIY'],
              coeffs=[1.+0.j])
==TERM== 0.5j [X0 Y2]
pauli string ((0, 'X'), (2, 'Y'))
AA
--qiskit_op-1 SparsePauliOp(['X'],
              coeffs=[1.+0.j])
AA
--qiskit_op-1 SparsePauliOp(['YIX'],
              coeffs=[1.+0.j])
BB True
0
coefficient 0.5j
qiskit_op SparsePauliOp(['YIX'],
              coeffs=[1.+0.j])


In [116]:
from src.sparse_tools import get_sparse_operator
from openfermion import jordan_wigner


r = 0.742
molecule = create_h2(r)

print(molecule.fci_energy)

# print(molecule.get_molecular_hamiltonian())
hamiltonian = molecule.get_molecular_hamiltonian()
# print(hamiltonian)
hamiltonian = jordan_wigner(hamiltonian)
print(type(hamiltonian))
print(hamiltonian)
# hamiltonian = get_sparse_operator(molecule.get_molecular_hamiltonian())
# print(hamiltonian)
hamiltonian_qiskit = to_qiskit_operator(hamiltonian)
print(hamiltonian_qiskit)

-1.1372633383635353
<class 'openfermion.ops.operators.qubit_operator.QubitOperator'>
-0.0996319255760924 [] +
-0.04533060139114829 [X0 X1 Y2 Y3] +
0.04533060139114829 [X0 Y1 Y2 X3] +
0.04533060139114829 [Y0 X1 X2 Y3] +
-0.04533060139114829 [Y0 Y1 X2 X3] +
0.17110568429296397 [Z0] +
0.1685935681755208 [Z0 Z1] +
0.1205103673498828 [Z0 Z2] +
0.1658409687410311 [Z0 Z3] +
0.17110568429296397 [Z1] +
0.1658409687410311 [Z1 Z2] +
0.1205103673498828 [Z1 Z3] +
-0.22250984629381015 [Z2] +
0.1743208433444256 [Z2 Z3] +
-0.22250984629381015 [Z3]
N qubits:  4
==TERM== -0.0996319255760924 []
n 4
empty qiskit term SparsePauliOp(['IIII'],
              coeffs=[-0.09963193+0.j])
==TERM== 0.17110568429296397 [Z0]
pauli string ((0, 'Z'),)
AA
--qiskit_op-1 SparsePauliOp(['Z'],
              coeffs=[1.+0.j])
BB True
3
--I SparsePauliOp(['I'],
              coeffs=[1.+0.j])
--qiskit_op-2 SparsePauliOp(['Z'],
              coeffs=[1.+0.j])
--I SparsePauliOp(['I'],
              coeffs=[1.+0.j])
--qiskit_op-2 S

In [120]:
print(type(hamiltonian_qiskit))

commuted_hamiltonian = hamiltonian_qiskit.group_commuting(qubit_wise=True)
print(len(commuted_hamiltonian))
print(commuted_hamiltonian)

<class 'qiskit.quantum_info.operators.symplectic.sparse_pauli_op.SparsePauliOp'>
5
[SparsePauliOp(['YYXX', 'IIII'],
              coeffs=[-0.0453306 +0.j, -0.09963193+0.j]), SparsePauliOp(['XYYX'],
              coeffs=[0.0453306+0.j]), SparsePauliOp(['YXXY'],
              coeffs=[0.0453306+0.j]), SparsePauliOp(['XXYY'],
              coeffs=[-0.0453306+0.j]), SparsePauliOp(['IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[ 0.17110568+0.j,  0.17110568+0.j, -0.22250985+0.j, -0.22250985+0.j,
  0.16859357+0.j,  0.12051037+0.j,  0.16584097+0.j,  0.16584097+0.j,
  0.12051037+0.j,  0.17432084+0.j])]


In [239]:
import numpy as np

# Define Pauli matrices
eigen_I = np.array([1, 1])
eigen_X = np.array([1, -1])
eigen_Y = np.array([1, -1])
eigen_Z = np.array([1, -1])

# Map string characters to Pauli matrices
pauli_dict = {'I': eigen_I, 'X': eigen_X, 'Y': eigen_Y, 'Z': eigen_Z}

def get_eigenvalues(pauli_strings):
    eigen_vals = 1
    
    for pauli in reversed(pauli_strings):
        # Start with identity matrix
        # print(pauli)
        # print(pauli_dict[pauli])
        eigen_vals = np.kron(eigen_vals, pauli_dict[pauli])
        
        # Apply the corresponding Pauli matrix for each qubit
        # for char in pauli:
        #     op = np.kron(op, pauli_dict[char])
        
        # eigen_vals.append(op)
    
    return eigen_vals

# print("Eigen value result:", get_eigenvalues('IZIIZZXY'))

In [ ]:


def get_probability_distribution(counts, NUM_SHOTS, N):
    # Generate all possible N-qubit measurement outcomes
    all_possible_outcomes = [''.join(format(i, '0' + str(N) + 'b')) for i in range(2**N)]
    
    # Ensure all possible outcomes are in counts
    for k in all_possible_outcomes:
        if k not in counts.keys():
            counts[k] = 0
    
    # Sort counts by outcome
    sorted_counts = sorted(counts.items())
    
    # Calculate the probability distribution
    output_distr = [v[1] / NUM_SHOTS for v in sorted_counts]
    
    return output_distr

In [245]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit_ibm_runtime import SamplerV2

X = Pauli("X")
Z = Pauli("Z")
I = Pauli("I")
Y = Pauli("Y")
sampler = SamplerV2(backend=AerSimulator())
shots = 500

N_qubits = commuted_hamiltonian[0].num_qubits
qc = QuantumCircuit(N_qubits)
qc.h(np.arange(N_qubits))

qc_updates = []
energy = 0

for cliques in commuted_hamiltonian:
    # print(cliques)
    qc_update = qc.copy()
    i = 0
    for pauli in cliques[0].paulis[0]:
        # print(pauli, i)
        if (pauli == Y):
            qc_update.sdg(i)
            qc_update.h(i)
        elif (pauli == X):
            qc_update.h(i)
        i += 1
    qc_update.measure_all()
    qc_updates.append(qc_update) 

    # Circuit Obtained
    job = sampler.run([(qc_update)], shots=shots)
    counts = job.result()[0].data.meas.get_counts()
    # print(counts)

    probs = get_probability_distribution(counts, shots, N_qubits)
    # print(probs, "\n")


    for pauli_string in cliques:
        # print(pauli_string.paulis)
        # print(pauli_string.coeffs)
        # print(type(pauli_string.paulis[0]))
        # print(type(pauli_string.coeffs[0]))
        # print(pauli_string.to_list()[0][0])
        # print(type(pauli_string.to_list()[0][0]))
        eigen_value = get_eigenvalues(pauli_string.to_list()[0][0])
        # print(eigen_value)

        res = np.dot(eigen_value, probs)*pauli_string.coeffs
        energy += res
        

    #     print("Result:", res)
    #     print("Energy:", energy)
        
    #     print("\n")

    # print("\n")

print(energy)


[-0.09526028+0.j]


In [166]:
print(qc_updates[4])

     
q_3: 
     
q_2: 
     
q_1: 
     
q_0: 
     


In [93]:
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
 
X = SparsePauliOp("X")
Z = SparsePauliOp("Z")
I = SparsePauliOp("I")
Y = SparsePauliOp("Y")
 
# build the evolution gate
operator = (Z ^ Z) - 0.1 * (X ^ I) + 0.1 * (I ^ I)
operator

SparsePauliOp(['ZZ', 'XI', 'II'],
              coeffs=[ 1. +0.j, -0.1+0.j,  0.1+0.j])